# Get Model Metrics
Used to get model performance metrics.

Requires saved models from either:
* `cnn_mfcc.ipynb`
* `cnn_mfcc_delta.ipynb`
* `cnn_raw.ipynb`

Requires data from:
* `generate_data_py.ipynb`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

In [2]:
# Define our CNNs
class mfcc_CNN(nn.Module):
    def __init__(self):
        super(mfcc_CNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(6784, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        # Convolutional layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        # Flatten the output into a 1D vector for use in the fully connected layers
        size = x.size()[1:]  #all size except batchSize
        numFeatures = 1
        for s in size:
            numFeatures *= s
        x = x.view(-1, numFeatures)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        #x = torch.sigmoid(self.fc2(x))
        x = self.fc2(x)
        return x
        
class mfcc_d_CNN(nn.Module):
    def __init__(self):
        super(mfcc_d_CNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(6784, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        # Convolutional layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        # Flatten the output into a 1D vector for use in the fully connected layers
        size = x.size()[1:]  #all size except batchSize
        numFeatures = 1
        for s in size:
            numFeatures *= s
        x = x.view(-1, numFeatures)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        #x = torch.sigmoid(self.fc2(x))
        x = self.fc2(x)
        return x
        
class raw_CNN(nn.Module):
    def __init__(self):
        super(raw_CNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv1d(1,  32,  kernel_size=10, padding=1)
        self.conv2 = nn.Conv1d(32, 64,  kernel_size=5, padding=1)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        
        # Pooling layer
        self.pool = nn.AvgPool1d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(3527808, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        # Convolutional layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        # Flatten the output for fully connected layers
        size = x.size()[1:]  #all size except batchSize
        numFeatures = 1
        for s in size:
            numFeatures *= s
        x = x.view(-1, numFeatures)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# load Model
mfcc_model_name       = os.path.join("models", "cnn_mfcc_model_testing.pth")
mfcc_delta_model_name = os.path.join("models", "cnn_mfcc_delta_model_testing.pth")
raw_model_name        = os.path.join("models", "cnn_raw_model_batch_100_epoch_5.pth")

# Load the state dictionary
state_dict_1 = torch.load(mfcc_model_name)
state_dict_2 = torch.load(mfcc_delta_model_name)
state_dict_3 = torch.load(raw_model_name)

# Create an instance of model
model_mfcc   = mfcc_CNN()
model_mfcc_d = mfcc_d_CNN()
model_raw    = raw_CNN()

# Load the state dictionary into the model
model_mfcc.load_state_dict(state_dict_1)
model_mfcc_d.load_state_dict(state_dict_2)
model_raw.load_state_dict(state_dict_3)

# Set the model in evaluation mode
model_mfcc.eval()
model_mfcc_d.eval()
model_raw.eval()

raw_CNN(
  (conv1): Conv1d(1, 32, kernel_size=(10,), stride=(1,), padding=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(1,))
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (fc1): Linear(in_features=3527808, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [4]:
# load up mfcc data
mfcc_labels = np.load(os.path.join("data", "labels.npy"))
mfcc_data   = np.load(os.path.join("data", "mfcc_data.npy"))
print(mfcc_labels.shape)
print(mfcc_data.shape)

(1049,)
(1049, 431, 12)


In [5]:
# Convert data to tensor compliant dataset
batchSize = 30;
mfccs_train, mfccs_test, labels_train, labels_test = train_test_split(mfcc_data, mfcc_labels, test_size=0.2, random_state=100)

mfccs_train_tensor  = torch.tensor(mfccs_train).unsqueeze(1)
mfccs_test_tensor   = torch.tensor(mfccs_test).unsqueeze(1)
labels_train_tensor = torch.tensor(labels_train)
labels_test_tensor  = torch.tensor(labels_test)

train_dataset = TensorDataset(mfccs_train_tensor, labels_train_tensor)
test_dataset  = TensorDataset(mfccs_test_tensor, labels_test_tensor)

train_loader = DataLoader(train_dataset, batch_size = batchSize, shuffle = True)
test_loader  = DataLoader(test_dataset, batch_size = batchSize, shuffle = False)

In [6]:
# Evaluate mfcc
model_mfcc.eval()
totalLabels = []
totalPredicted = []
with torch.no_grad():
    for t_inputs, t_labels in test_loader:
        outputs = model_mfcc(t_inputs)
        _, predicted = torch.max(outputs.data, 1)
        totalLabels += t_labels
        totalPredicted += predicted
print(f"Test Accuracy: {accuracy_score(totalLabels, totalPredicted)}")
print(f"Test Precision: {precision_score(totalLabels, totalPredicted)}")
print(f"F1: {f1_score(totalLabels, totalPredicted)}")
print(f"Recall: {recall_score(totalLabels, totalPredicted)}")
print(f"Confusion Matrix: {confusion_matrix(totalLabels, totalPredicted)}")

Test Accuracy: 0.9380952380952381
Test Precision: 0.9603960396039604
F1: 0.9371980676328503
Recall: 0.9150943396226415
Confusion Matrix: [[100   4]
 [  9  97]]


In [7]:
# load up mfcc_d data
mfcc_labels = np.load(os.path.join("data", "labels.npy"))
mfcc_data   = np.load(os.path.join("data", "mfcc_delta_data.npy"))
print(mfcc_labels.shape)
print(mfcc_data.shape)

(1049,)
(1049, 3, 431, 12)


In [8]:
# Convert data to tensor compliant dataset
batchSize = 30;
mfccs_train, mfccs_test, labels_train, labels_test = train_test_split(mfcc_data, mfcc_labels, test_size=0.2, random_state=100)

mfccs_train_tensor  = torch.tensor(mfccs_train)
mfccs_test_tensor   = torch.tensor(mfccs_test)
labels_train_tensor = torch.tensor(labels_train)
labels_test_tensor  = torch.tensor(labels_test)

print(mfccs_train_tensor.shape)

train_dataset = TensorDataset(mfccs_train_tensor, labels_train_tensor)
test_dataset  = TensorDataset(mfccs_test_tensor, labels_test_tensor)

train_loader = DataLoader(train_dataset, batch_size = batchSize, shuffle = True)
test_loader  = DataLoader(test_dataset, batch_size = batchSize, shuffle = False)

torch.Size([839, 3, 431, 12])


In [9]:
# Evaluate mfcc with delta features
model_mfcc_d.eval()
totalLabels = []
totalPredicted = []
with torch.no_grad():
    for t_inputs, t_labels in test_loader:
        outputs = model_mfcc_d(t_inputs)
        _, predicted = torch.max(outputs.data, 1)
        totalLabels += t_labels
        totalPredicted += predicted
print(f"Test Accuracy: {accuracy_score(totalLabels, totalPredicted)}")
print(f"Test Precision: {precision_score(totalLabels, totalPredicted)}")
print(f"F1: {f1_score(totalLabels, totalPredicted)}")
print(f"Recall: {recall_score(totalLabels, totalPredicted)}")
print(f"Confusion Matrix: {confusion_matrix(totalLabels, totalPredicted)}")

Test Accuracy: 0.9476190476190476
Test Precision: 0.9279279279279279
F1: 0.9493087557603687
Recall: 0.9716981132075472
Confusion Matrix: [[ 96   8]
 [  3 103]]


In [10]:
# load up raw data
raw_labels = np.load(os.path.join("data", "labels.npy"))
raw_data = np.load(os.path.join("data", "raw_data.npy"))
print(raw_labels)
print(raw_labels.shape)
print(raw_data.shape)

[1 1 1 ... 0 0 0]
(1049,)
(1049, 220500)


In [11]:
# Convert data to tensor compliant dataset
batchSize = 100
raw_train, raw_test, labels_train, labels_test = train_test_split(raw_data, raw_labels, test_size=0.2, random_state=100)

raw_train_tensor    = torch.tensor(raw_train).unsqueeze(1)
raw_test_tensor     = torch.tensor(raw_test).unsqueeze(1)
labels_train_tensor = torch.tensor(labels_train)
labels_test_tensor  = torch.tensor(labels_test)

train_dataset = TensorDataset(raw_train_tensor, labels_train_tensor)
test_dataset  = TensorDataset(raw_test_tensor, labels_test_tensor)

train_loader  = DataLoader(train_dataset, batch_size = batchSize, shuffle = True)
test_loader   = DataLoader(test_dataset, batch_size = batchSize, shuffle = False)

In [12]:
# Evaluate
model_raw.eval()
correct = 0
total = 0
totalLabels    = []
totalPredicted = []
with torch.no_grad():
    for t_inputs, t_labels in test_loader:
        outputs = model_raw(t_inputs)
        _, predicted = torch.max(outputs.data, 1)
        totalLabels    += t_labels
        totalPredicted += predicted
print(f"Test Accuracy: {accuracy_score(totalLabels, totalPredicted)}")
print(f"Test Precision: {precision_score(totalLabels, totalPredicted)}")
print(f"F1: {f1_score(totalLabels, totalPredicted)}")
print(f"Recall: {recall_score(totalLabels, totalPredicted)}")
print(f"Confusion Matrix: {confusion_matrix(totalLabels, totalPredicted)}")

Test Accuracy: 0.9095238095238095
Test Precision: 0.9223300970873787
F1: 0.9090909090909091
Recall: 0.8962264150943396
Confusion Matrix: [[96  8]
 [11 95]]
